In [1]:
from os import listdir
import re
import chardet
import pandas as pd
import datetime
from city import City
import mysql.connector
from mysql.connector.cursor import MySQLCursorPrepared

In [2]:
now = datetime.datetime.now()

In [3]:
df = pd.read_csv("worldcitiespop.csv")
df.head(1)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6,NaN,42.483333,1.466667


In [4]:
df.count()[0]

3173958

In [5]:
df = df[df["AccentCity"].notna()]
df = df[df["Latitude"].notna()]
df = df[df["Longitude"].notna()]
df.count()[0]

3173958

In [6]:
df = df.sort_values(["Population"], ascending=False)
df = df.drop_duplicates(subset=["AccentCity"], keep="first")

In [7]:
all_cities = set(df["AccentCity"])

In [8]:
def extract_cities(file_text):
    word_list = re.sub("[^\w]", " ",  file_text).split()
    return list(all_cities.intersection(word_list))

In [9]:
def extract_details(file_text):
    re_patterns = [
        "[ ]{0,4}Title: (.+)\n\n[ ]{0,4}Author: ([^\n]+)\n",
        "  We need your donations.\n\n\n([^\n]+)\n\nby ([^\n]+)\n\n",
        "\n\nTitle: (.+)\nAuthor: (.+)\nRelease Date: ",
        "\n\n\n\n\n\n[\d]{4}\n\n()\n\nby ()\n\n\nDramatis Personae",
    ]

    title = None
    author = None

    for pattern in re_patterns:
        try:
            result = re.search(pattern, file_text, re.DOTALL)
            title = result.group(1)
            author = result.group(2)
            break
        except:
            continue
    
    return title, author

In [ ]:
txt_files = [f for f in listdir() if f.endswith(".txt")]
#txt_files = [txt_files[0]]
success = 0
fail = 0

encoding_failed = []
regex_failed = []

books = []

for file_name in txt_files:
    
    with open(file_name, 'r') as file:
        
        try:
            file_text = file.read()
            
            title, author = extract_details(file_text)
            
            if author == None or title == None:
                print(file_name + "******REGEX*************************************************")
                fail += 1
                regex_failed.append(file_name)
                continue
            
            print(file_name)
            print("TITLE: " + title)
            print("AUTHOR: " + author + "\n")
            
            cities = extract_cities(file_text)
            
            
            success += 1
                
        except:
            print(file_name + "********ENCODING***********************************************")
            fail += 1
            encoding_failed.append(file_name)

print("Succes: " + str(success))
print("Fail: " + str(fail))
print("regex_errors: " + str(len(regex_failed)))
print("encoding_error: " + str(len(encoding_failed)))
print("Total: " + str(len(txt_files)))

In [ ]:
regex_failed[:10]

In [ ]:
encoding_failed[:10]

In [ ]:
time_diff = datetime.datetime.now() - now
time_diff.total_seconds() / 60

In [ ]:
c = City(1, "Berlin", 2, 2)

print(c.stringify())

In [10]:
con = mysql.connector.connect(
    host="localhost",
    user="dbexamuser",
    database="examdb",
    passwd="dbExam.group7",
    use_pure=True,
)
con.autocommit = False
mycursor = con.cursor(cursor_class=MySQLCursorPrepared)
print(con) 

In [11]:
def insert_cities():
    now = datetime.datetime.now()
    large_sql = ""
    counter = 1
    interval_counter = 0
    interval = 1000
    
    for i, row in df.iterrows():
        city_name = row["AccentCity"]
        city_long = row["Longitude"]
        city_lat = row["Latitude"]
        #print(city_name)
        #print(city_long)
        #print(city_lat)
        sql = """INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));"""
        #sql = "INSERT INTO authors (name) VALUES ('bob');"
        #large_sql += sql
        
        insert_values = (city_name, city_lat, city_long)
        
        #print(sql % insert_values)
        #print("city name: ")
        #print(type(city_name))
        #print(counter)
        
        mycursor.execute(sql, insert_values)
        #print(mycursor.statement)
        
        if (counter >= interval_counter + interval):
            print(mycursor.statement)
            con.commit()
            interval_counter += interval
            print(counter)
            
        counter += 1
        
    #print(large_sql)
    #mycursor.execute(large_sql, multi = True)
    con.commit()
    
    time_diff = datetime.datetime.now() - now
    print(time_diff.total_seconds())

insert_cities()

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
3000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
4000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
5000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
6000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
7000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
8000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
9000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
10000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
11000
INSERT I

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
92000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
93000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
94000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
95000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
96000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
97000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
98000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
99000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
100000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
101000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
102

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
182000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
183000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
184000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
185000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
186000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
187000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
188000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
189000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
190000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
191000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
272000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
273000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
274000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
275000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
276000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
277000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
278000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
279000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
280000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
281000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
362000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
363000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
364000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
365000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
366000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
367000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
368000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
369000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
370000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
371000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %

...

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1971000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1972000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1973000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1974000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1975000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1976000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1977000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1978000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1979000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
1980000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Po

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2060000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2061000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2062000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2063000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2064000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2065000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2066000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2067000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2068000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2069000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Po

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2149000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2150000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2151000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2152000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2153000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2154000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2155000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2156000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2157000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2158000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Po

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2238000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2239000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2240000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2241000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2242000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2243000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2244000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2245000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2246000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2247000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Po

INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2327000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2328000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2329000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2330000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2331000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2332000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2333000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2334000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2335000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Point(%s , %s));
2336000
INSERT INTO examdb.cities (name, location) 
            VALUES (%s, Po

In [12]:
print("done")

done


In [13]:
con.close()
mycursor.close()

In [ ]:
#df[df["Latitude"] == 42.5833333]